In [ ]:
import pandas as pd
import requests
from io import StringIO
#%conda install lxml #comment this out after done
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
    
url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"
res = requests.get(url, headers=headers).text
res = StringIO(res) #pandas read_html needs a file-like object, so we use StringIO to convert the string to a file-like object


df = pd.read_html(res)
df

2 channel Terms of Service accepted
Channels:
 - defaults
Platform: osx-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/claredasso/anaconda3/envs/python-class

  added / updated specs:
    - lxml


The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/osx-64::libxslt-1.1.41-ha190046_0 
  lxml               pkgs/main/osx-64::lxml-5.3.0-py313h8defbaf_1 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


[    0                                                  1
 0 NaN  This section needs additional citations for ve...,
    Rank              Player     Club                  Country and other  \
    Rank              Player   League  Cup Continental Country and other   
 0     1   Cristiano Ronaldo   590[b]   57         172               143   
 1     2       Lionel Messi*   553[c]   71         157               115   
 2     3               Pelé*   604[d]   49          26                83   
 3     4             Romário   545[e]   93          54                64   
 4     5       Ferenc Puskás   516[f]   69          56                84   
 5     6        Josef Bican*   515[g]  137          38                32   
 6     7  Robert Lewandowski   423[h]   62         117                88   
 7     8        Jimmy Jones*   330[i]  286          14                 9   
 8     9        Gerd Müller*   405[j]   92          69                68   
 9    10       Joe Bambrick*   347[k]  253     

In [2]:
df = df[3]

In [ ]:
#df.to_csv("./data/top500goals.csv", index=False)
#comment above out when done

In [5]:
df = pd.read_csv("./data/top500goals.csv")
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989+,582,1.70,1924–1951
1,2,Cristiano Ronaldo,975,1337,0.73,2002–present
2,3,Josef Bican,950+,624,1.52,1930–1957
3,4,Ronnie Rooke,934+,1030,0.91,1929–1961
4,5,Lionel Messi,925,1194,0.77,2003–present


In [10]:
import random, math

df["Career span"] = df["Career span"].str.split("–").str[0]

In [7]:
df["Career span"].isna().sum()

np.int64(0)

In [ ]:
#df.loc[~df['Goals'].str.isnumeric()] #comment out after running
df["Goals"] = [item.replace("+", "") for item in df["Goals"]]
df["Goals"]

0     989
1     975
2     950
3     934
4     925
     ... 
78    504
79    504
80    503
81    502
82    500
Name: Goals, Length: 83, dtype: object

In [11]:
df["Goals"] = df["Goals"].astype(int)
df["Career span"] = df["Career span"].astype(int)
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989,582,1.70,1924
1,2,Cristiano Ronaldo,975,1337,0.73,2002
2,3,Josef Bican,950,624,1.52,1930
3,4,Ronnie Rooke,934,1030,0.91,1929
4,5,Lionel Messi,925,1194,0.77,2003


In [12]:
df["Matches"] = df["Matches"].str.replace("+", "").astype(int)
df.describe()

,Rank,Goals,Matches,Ratio,Career span
count,83.000000,83.000000,83.000000,83.000000,83.00000
mean,42.000000,606.361446,724.915663,0.886024,1945.53012
std,24.103942,122.085394,193.411996,0.267563,26.39306
min,1.000000,500.000000,324.000000,0.530000,1891.00000
25%,21.500000,521.000000,590.500000,0.695000,1927.50000
50%,42.000000,564.000000,711.000000,0.860000,1938.00000
75%,62.500000,631.000000,836.500000,0.960000,1961.00000
max,83.000000,989.000000,1337.000000,1.700000,2010.00000


In [13]:
from datetime import datetime, timedelta
rows = []
columns = ["name", "date", "goals", "assists"]

for row in df.itertuples():
    name = row.Player
    start_date = f"{row._6}-08-01"
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    goal_ratio = row.Ratio
    goal_sd = random.random()
    goal_sd = goal_sd if goal_sd < 0.7 else 0.7
    assist_sd = 1 - goal_sd
    assist_ratio = 1- goal_ratio
    
    for i in range(row.Matches):
        name = name
        match_date = timedelta(days=3 * i) + start_date
        goals = random.normalvariate(goal_ratio, goal_sd)
        assists = random.normalvariate(assist_ratio, assist_sd)
        goals = round(goals) if goals > 0 else 0
        assists = round(assists) if assists > 0 else 0
        row = [name, match_date, goals, assists]
        rows.append(row)
games_df = pd.DataFrame(data=rows, columns=columns)
games_df.head()

,name,date,goals,assists
0,Erwin Helmchen,1924-08-01,1,0
1,Erwin Helmchen,1924-08-04,2,0
2,Erwin Helmchen,1924-08-07,1,0
3,Erwin Helmchen,1924-08-10,2,0
4,Erwin Helmchen,1924-08-13,1,0


In [14]:
games_df.shape

(60168, 4)

In [15]:
games_df.sample(10) #random sample of 10 rows from the dataframe

,name,date,goals,assists
49784,Giorgio Chinaglia,1964-04-07,1,0
43522,Alfredo Di Stéfano,1946-12-06,1,1
1889,Cristiano Ronaldo,2013-04-26,1,0
17582,Stan Mortensen,1943-08-02,1,1
58786,Albert de Cleyn,1938-05-10,1,0
46584,W. G. Richardson,1930-08-15,0,1
8695,Pelé,1956-08-25,1,0
18833,Joe Smith,1913-12-27,0,1
40798,Tommy Dickson,1949-05-08,1,0
19477,Frederick Roberts,1925-11-10,1,0


In [16]:
fake_df = games_df.groupby("name").agg(
    matches=("name", "count"),
    goals=("goals", "sum"),
    assists=("assists", "sum"),
    g_ratio=("goals", "mean"),
    a_ratio=("assists", "mean")  
).reset_index().sort_values("goals", ascending=False).reset_index(drop=True)

fake_df.head(10)

,name,matches,goals,assists,g_ratio,a_ratio
0,Cristiano Ronaldo,1337,1051,318,0.786088,0.237846
1,Ronnie Rooke,1030,1018,338,0.988350,0.328155
2,Erwin Helmchen,582,1011,2,1.737113,0.003436
3,Lionel Messi,1194,957,215,0.801508,0.180067
4,Josef Bican,624,956,83,1.532051,0.133013
5,Abe Lenstra,850,850,332,1.000000,0.390588
6,Pelé,851,830,85,0.975323,0.099882
7,Romário,1003,827,171,0.824526,0.170489
8,Jimmy Jones,760,823,124,1.082895,0.163158
9,Ferenc Deák,515,803,3,1.559223,0.005825


In [18]:
for i in range(10):
    print(f"{i} df: {df.loc[i,["Player", "Goals"]]} | {i} fake: {fake_df.loc[i,["name", "goals"]]}")

0 df: Player    Erwin Helmchen
Goals                989
Name: 0, dtype: object | 0 fake: name     Cristiano Ronaldo
goals                 1051
Name: 0, dtype: object
1 df: Player    Cristiano Ronaldo
Goals                   975
Name: 1, dtype: object | 1 fake: name     Ronnie Rooke
goals            1018
Name: 1, dtype: object
2 df: Player    Josef Bican
Goals             950
Name: 2, dtype: object | 2 fake: name     Erwin Helmchen
goals              1011
Name: 2, dtype: object
3 df: Player    Ronnie Rooke
Goals              934
Name: 3, dtype: object | 3 fake: name     Lionel Messi
goals             957
Name: 3, dtype: object
4 df: Player    Lionel Messi
Goals              925
Name: 4, dtype: object | 4 fake: name     Josef Bican
goals            956
Name: 4, dtype: object
5 df: Player    Jimmy Jones
Goals             840
Name: 5, dtype: object | 5 fake: name     Abe Lenstra
goals            850
Name: 5, dtype: object
6 df: Player    Ferenc Puskás
Goals               802
Name: 6, dtype